## Домашнее задание 4 — NER

### 1. Установка и импорт библиотек

### СЕКЦИЯ 1: ПЕРВОЕ NER ОБУЧЕНИЕ

**Выполните все ячейки этой секции до ячейки с рестартом ядра**


In [1]:
# Установка необходимых библиотек
%pip install -r requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... Python(42874) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 5.7 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 kB 11.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.8 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 17.1 MB/s  0:00:01 eta 0:00:01
done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16250 sha256=3d5fa5dc4265793cfdeb5c75e0837042f145919b7691dd7f6173c7afc6c945a1
  Stored in directory: /Users/bitcoin/Library/Caches/pip/wheels/e2/a5/92/2c80d1928733611c2747a9820e1324a6835524d9411510c142
Successfully built

In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


#### Импорт необходимых библиотек

In [2]:
import gc
import random
import zipfile

import numpy as np
import pandas as pd
import torch
import evaluate
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from IPython.display import display, HTML

from datasets import Dataset, concatenate_datasets
from corus import load_ne5
import corus.sources.ne5 as ne5
import razdel
from razdel.substring import Substring
from dataclasses import dataclass

from transformers import (
    AutoModelForMaskedLM,
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
    pipeline
)

# Установка random seed для воспроизводимости результатов
RANDOM_SEED = 777
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

print("RANDOM SEED WAS SET TO:", RANDOM_SEED)

/home/bitcoin/ITMO-DL-NLP/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RANDOM SEED WAS SET TO: 777


### 2. Загрузка данных

In [4]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip
!rm collection5.zip

--2025-09-26 12:00:22--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 64:ff9b::5fb5:e6b5
Connecting to www.labinform.ru (www.labinform.ru)|64:ff9b::5fb5:e6b5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  2.29MB/s    in 0.8s    

2025-09-26 12:00:23 (2.29 MB/s) - ‘collection5.zip’ saved [1899530/1899530]

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Co

In [3]:
# Загрузка набора данных Collection5 с использованием метода load_ne5
print("Загружаем данные Collection5...")

dir = 'Collection5'
records = list(load_ne5(dir))
print(f"Загружено записей: {len(records)}")

# Посмотрим на структуру первых нескольких записей
print("\nПример записей:")
for i, record in enumerate(records[:5]):
    print(f"Запись {i+1}:")
    print(f"  Текст: {record.text[:100]}...")
    print(f"  Spans: {record.spans[:5] if record.spans else 'Нет'}")
    print()

Загружаем данные Collection5...
Загружено записей: 1000

Пример записей:
Запись 1:
  Текст: Галушка приступил к работе по ликвидации последствий наводнения (ДФО)

В район бедствия в Хабаровс...
  Spans: [Ne5Span(index='T1', type='PER', start=0, stop=7, text='Галушка'), Ne5Span(index='T2', type='LOC', start=65, stop=68, text='ДФО'), Ne5Span(index='T3', type='LOC', start=92, stop=108, text='Хабаровский край'), Ne5Span(index='T4', type='LOC', start=152, stop=168, text='Дальнего Востока'), Ne5Span(index='T5', type='PER', start=169, stop=186, text='Александр Галушка')]

Запись 2:
  Текст: 
Ульяновские депутаты утвердили заместителей главы администрации города

Депутаты Ульяновской гор...
  Spans: [Ne5Span(index='T1', type='ORG', start=85, stop=111, text='Ульяновской городской думы'), Ne5Span(index='T2', type='LOC', start=180, stop=190, text='Ульяновска'), Ne5Span(index='T3', type='MEDIA', start=216, stop=225, text='ИА REGNUM'), Ne5Span(index='T4', type='ORG', start=241, stop=244, text='УГД'

In [4]:
# Определение классов для работы с данными
@dataclass
class Ne5Span:
    start: int
    stop: int
    type: str

@dataclass
class LabelledText:
    text: str
    entities: list[Ne5Span]


In [5]:
def parse_records(records: list) -> tuple[list[LabelledText], dict[str, int]]:
    """Parse records into LabelledText objects and create a category map."""
    labelled_texts = []
    cats_map = {}

    for annot in records:
        entities = []
        
        for ent in annot.spans:
            new_ent = Ne5Span(
                start=ent.start,
                stop=ent.stop,
                type=ent.type
            )

            entities.append(new_ent)
            if new_ent.type not in cats_map:
                cats_map[new_ent.type] = len(cats_map)

        entities.sort(key=lambda x: x.start)
        lab_txt = LabelledText(
            text=annot.text,
            entities=entities
        )
        labelled_texts.append(lab_txt)

    return labelled_texts, cats_map

In [6]:
labelled_text, categories_map = parse_records(records)

print(f'Всего записей: {len(labelled_text)}')
print('Категории объектов:', categories_map)

Всего записей: 1000
Категории объектов: {'PER': 0, 'LOC': 1, 'MEDIA': 2, 'GEOPOLIT': 3, 'ORG': 4}


In [7]:
# Разбивка на train/test части (80/20)
train_data, test_data = train_test_split(labelled_text, test_size=0.2, random_state=RANDOM_SEED)

print(f"Train size: {len(train_data)}")
print(f"Test size: {len(test_data)}")


Train size: 800
Test size: 200


### 3. Токенизация датасета

In [8]:
# Токенизация и выравнивание меток
def tokenize_rus(text: str):
    """Implement word-wise tokenization for russian language."""
    return list(razdel.tokenize(text))

In [9]:
def symbol_bio_to_word_bio(labelled_text: LabelledText, subwords: list[Substring]) -> list[str]:
    """Transforms symbol-wise annotation to word-wise BIO notation for NER.

    Args:
        labelled_text: A LabelledText object containing the original text and
            symbol-wise entity annotations.
        subwords: A list of Substring objects representing the tokenized text.

    Returns:
        A list of strings representing BIO tags in the format:
        - "B-CAT" for beginning of an entity of category CAT
        - "I-CAT" for continuation of an entity of category CAT
        - "O" for tokens outside any entity
    """
    bio_tags = ["O"] * len(subwords)
    entities = sorted(labelled_text.entities, key=lambda x: x.start)
    
    for entity in entities:
        overlapping = []
        for i, subword in enumerate(subwords):
            if (subword.start < entity.stop and subword.stop > entity.start):
                overlapping.append(i)

        for j, idx in enumerate(overlapping):
            bio_tags[idx] = f"B-{entity.type}" if j == 0 else f"I-{entity.type}"

    return bio_tags

In [10]:
def align_labels_with_tokens(labels: list[str], word_ids: list[str]) -> list[str]:
    """Aligns word-level BIO labels to token-level labels accounting for wordpiece tokenization.
    
    Args:
        labels: List of BIO worl-level labels.
        word_ids: List mapping each token to its original word index (from tokenizer.word_ids()).
    
    Returns:
        List of aligned token-level BIO labels.
    """
    new_labels = []
    cur_word_id = None
    
    for word_id in word_ids:
        if word_id is None:
            new_labels.append('Ignored')  # special token

        elif word_id != cur_word_id:
            cur_word_id = word_id
            new_labels.append(labels[word_id])

        else:
            label = labels[word_id]
            if label.startswith("B-"):
                label = f"I-{label[2:]}"

            new_labels.append(label)

    return new_labels

In [11]:
class LabelsTokenizerAligner:
    def __init__(self, bio_labels_to_idx: dict[str, int], tokenizer):
        self.bio_labels_to_idx = bio_labels_to_idx
        self.tokenizer = tokenizer

    def _tokenize_and_align_labels(self, examples: Dataset):
        """Tokenizes input text and aligns word-level BIO labels with subword tokens.
    
        Args:
            examples (Dataset): A Hugging Face `Dataset` object containing:
                - `words`: List of words.
                - `labels_bio`: List of word-level BIO labels.
            tokenizer (PreTrainedTokenizer): Tokenizer (e.g., `BertTokenizer`) with subword tokenization.

        Returns:
            (Dataset): A Hugging Face `Dataset` object.
        """
        tokenized_inputs = self.tokenizer(
            examples["words"], 
            truncation=True, 
            max_length=512,  # Явно устанавливаем максимальную длину для предотвращения CUDA OOM
            padding=False,  # Позже используем data_collator для паддинга
            is_split_into_words=True
        )

        aligned_labels = []
        for sample_idx, bio_labels in enumerate(examples["bio_labels"]):
            word_ids = tokenized_inputs.word_ids(batch_index=sample_idx)
            bio_labels_aligned = align_labels_with_tokens(bio_labels, word_ids)
            aligned_labels.append([self.bio_labels_to_idx[bla] for bla in bio_labels_aligned])

        tokenized_inputs["labels"] = aligned_labels
        return tokenized_inputs

    def __call__(self, examples: Dataset):
        return self._tokenize_and_align_labels(examples)

In [12]:
def make_tokenized_dataset(ds_ner: list, bio_labels_to_idx: dict[str, int], tokenizer) -> Dataset:
    """Make tokenized dataset ready for batching.

    Makes(Dataset): a Hugging Face `Dataset` object containing model inputs:
        input_ids, token_type_ids, and an attention_mask via the steps:
        1) Represent raw dataset as python dictionary of words and bio_labels.
        2) Split sentences into words and convert symbol-wise labeling into word-wise BIO format.
        3) Tokenize words and align the labels with sub-tokens level.

    Args:
        ds_ner: List of (LabelledText) objects.
        bio_labels_to_idx: Map for BIO labels to int including 'Ignored' key for special tokens.
        tokenizer (PreTrainedTokenizer): Tokenizer (e.g., `BertTokenizer`) with subword tokenization.

    Returns:
        (Dataset): A tokenized Hugging Face `Dataset` object.
    """
    ds_dict = {"words": [], "bio_labels": []}

    for sample in ds_ner:
        words_subs = tokenize_rus(sample.text)
        words = [ws.text for ws in words_subs]
        bio_labels = symbol_bio_to_word_bio(sample, words_subs)
        ds_dict["words"].append(words)
        ds_dict["bio_labels"].append(bio_labels)

    dataset = Dataset.from_dict(ds_dict)
    labels_tokenizer_aligner = LabelsTokenizerAligner(bio_labels_to_idx, tokenizer)

    tokenized_dataset = dataset.map(             
        labels_tokenizer_aligner,
        batched=True,
        remove_columns=['words', 'bio_labels']
    )
    
    return tokenized_dataset

In [13]:
bio_labels_to_idx = {'O': 0}

for cat in categories_map:
    for prefix in 'BI':
        bio_labels_to_idx[f'{prefix}-{cat}'] = len(bio_labels_to_idx)

bio_labels_to_idx['Ignored'] = -100

idx_to_bio_labels = {lbl: bio for bio, lbl in bio_labels_to_idx.items()}

bio_labels_to_idx

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-LOC': 3,
 'I-LOC': 4,
 'B-MEDIA': 5,
 'I-MEDIA': 6,
 'B-GEOPOLIT': 7,
 'I-GEOPOLIT': 8,
 'B-ORG': 9,
 'I-ORG': 10,
 'Ignored': -100}

In [14]:
!nvidia-smi

Fri Sep 26 16:15:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.20             Driver Version: 570.133.20     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           Off |   00000000:81:00.0 Off |                    0 |
| N/A   30C    P0             36W /  250W |    6534MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
# device for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Используется устройство: {device}")


Используется устройство: cuda:0


In [16]:
# Подготавливаем tokenizer и датасеты
model_checkpoint = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [17]:
# Пересоздаем датасеты с исправленной токенизацией (max_length=512)
print("Создаем токенизированные датасеты с max_length=512...")

ds_tokenized_test = make_tokenized_dataset(test_data, bio_labels_to_idx, tokenizer)
ds_tokenized_train = make_tokenized_dataset(train_data, bio_labels_to_idx, tokenizer)

print("Датасеты успешно созданы!")

Создаем токенизированные датасеты с max_length=512...


Map: 100%|██████████| 800/800 [00:00<00:00, 948.79 examples/s]

Датасеты успешно созданы!


In [18]:
print(f'test size: {len(ds_tokenized_test)}')
print(f'train size: {len(ds_tokenized_train)}')

test size: 200
train size: 800


In [19]:
# Анализ длин последовательностей для диагностики проблем с памятью
import numpy as np

def analyze_sequence_lengths(dataset, dataset_name):
    lengths = [len(item['input_ids']) for item in dataset]
    print(f"\n=== Анализ длин последовательностей для {dataset_name} ===")
    print(f"Количество последовательностей: {len(lengths)}")
    print(f"Минимальная длина: {min(lengths)}")
    print(f"Максимальная длина: {max(lengths)}")
    print(f"Средняя длина: {np.mean(lengths):.2f}")
    print(f"Медианная длина: {np.median(lengths):.2f}")
    print(f"Стандартное отклонение: {np.std(lengths):.2f}")
    
    # Проверим, есть ли последовательности, которые достигают максимальной длины
    max_len_count = sum(1 for l in lengths if l >= 510)  # близко к 512
    print(f"Последовательности длиной >= 510: {max_len_count}")
    
    return lengths

train_lengths = analyze_sequence_lengths(ds_tokenized_train, "train")
test_lengths = analyze_sequence_lengths(ds_tokenized_test, "test")



=== Анализ длин последовательностей для train ===
Количество последовательностей: 800
Минимальная длина: 28
Максимальная длина: 512
Средняя длина: 288.96
Медианная длина: 269.00
Стандартное отклонение: 123.03
Последовательности длиной >= 510: 93

=== Анализ длин последовательностей для test ===
Количество последовательностей: 200
Минимальная длина: 57
Максимальная длина: 512
Средняя длина: 280.00
Медианная длина: 265.00
Стандартное отклонение: 119.02
Последовательности длиной >= 510: 16


In [20]:
_test_idx = 0
print(f'raw test: {test_data[_test_idx].text}')
_tokenized_decoded = tokenizer.decode(ds_tokenized_test[_test_idx]['input_ids'], skip_special_tokens=True)
print(f"tok test: {_tokenized_decoded}")

raw test: Экс-глава X5 Retail Л.Хасис назначен вице-президентом Wal-Mart

Крупнейший в мире ритейлер - американский Wal-Mart - назначил экс-главу Х5 Retail Group Льва Хасиса на должность старшего вице-президента по международным операциям. Об этом сегодня сообщила официальный представитель американской компании, передает Reuters.

По ее словам, Л.Хасис будет работать в головном офисе Wal-Mart в Бентонвилле (штат Арканзас) и подчиняться напрямую главе Wal-Mart International Дагу Макмиллану. На новом посту экс-глава X5 будет отвечать за "интеграцию приобретаемых компаний, синергию в глобальных закупках и создание инновационных команд", отметила представитель компании.

Официально Л.Хасис вступит в должность с октября текущего года.

Л.Хасис сменит в новой должности Джона Адена, который занимал пост старшего вице-президента по международным операциям Wal-Mart с 2007г. по 2010г.

Напомним, Л.Хасис покинул X5 Retail в марте 2011г., чтобы заняться персональными проектами. На посту главного и

### 4. Дообучение модели rubert-tiny2

In [21]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [22]:
label2id = {label: idx for label, idx in bio_labels_to_idx.items() if idx >= 0}
id2label = {idx: label for label, idx in label2id.items()}

In [23]:
id2label

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-LOC',
 4: 'I-LOC',
 5: 'B-MEDIA',
 6: 'I-MEDIA',
 7: 'B-GEOPOLIT',
 8: 'I-GEOPOLIT',
 9: 'B-ORG',
 10: 'I-ORG'}

In [24]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
).to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
model.config.num_labels

11

In [26]:
metric = evaluate.load("seqeval")
label_names = list(label2id.keys()) 

def compute_metrics(eval_preds: tuple[np.ndarray, np.ndarray]) -> dict[str, float]:
    """
    Compute evaluation metrics for token classification (NER) using seqeval.

    Args:
        eval_preds (tuple): A tuple containing:
            - logits (np.ndarray): Model output logits of shape (batch_size, seq_len, num_labels)
            - labels (np.ndarray): Ground truth label ids of shape (batch_size, seq_len)

    Returns:
        dict: A dictionary with overall precision, recall, F1 score, and accuracy.
    """
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    metrics = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)

    return {
        "precision": metrics["overall_precision"],
        "recall": metrics["overall_recall"],
        "f1": metrics["overall_f1"],
        "accuracy": metrics["overall_accuracy"],
    }

In [27]:
# — оптимизатор AdamW (по умолчанию в HF Trainer)
# — lr=2e-5 — стандарт для дообучения BERT-подобных
# — batch_size=16 — укладывается в VRAM V100
# — epochs=25 — даёт стабильную сходимость на малых корпусах

training_args = TrainingArguments(
    output_dir="./ner_rubert_tiny2_dumps",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=25,
    weight_decay=0.01,
    seed=RANDOM_SEED,
    save_strategy="epoch",          # Сохраняем модель каждую эпоху
    save_total_limit=2,             # Храним только 2 последних checkpoint'а
    logging_steps=100,              # Логируем каждые 100 шагов
)

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tokenized_train,
    eval_dataset=ds_tokenized_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [29]:
print("Метрики до дообучения:")
trainer.evaluate(ds_tokenized_test)

Метрики до дообучения:


{'eval_loss': 2.516990900039673,
 'eval_model_preparation_time': 0.0018,
 'eval_precision': 0.0025687130747495505,
 'eval_recall': 0.026831785345717233,
 'eval_f1': 0.004688570707252858,
 'eval_accuracy': 0.05463930504847035,
 'eval_runtime': 2.1209,
 'eval_samples_per_second': 94.298,
 'eval_steps_per_second': 6.129}

In [30]:
# Очистка кэша GPU для освобождения памяти перед обучением
import gc

print("Очищаем кэш GPU...")
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    
# Принудительная очистка памяти Python
gc.collect()

print("Кэш очищен. Начинаем обучение...")


Очищаем кэш GPU...
Кэш очищен. Начинаем обучение...


In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Precision,Recall,F1,Accuracy
1,No log,0.827944,0.001800,0.000000,0.000000,0.000000,0.764591
2,1.033700,0.504461,0.001800,0.404519,0.310423,0.351279,0.852197
3,1.033700,0.377137,0.001800,0.493498,0.493498,0.493498,0.893581
4,0.394600,0.301762,0.001800,0.544061,0.615480,0.577571,0.919534
5,0.394600,0.258774,0.001800,0.579245,0.677399,0.624489,0.930775
6,0.259200,0.227866,0.001800,0.610484,0.713932,0.658168,0.937879
7,0.259200,0.204817,0.001800,0.645758,0.735191,0.687578,0.944012
8,0.198100,0.188789,0.001800,0.677513,0.769247,0.720472,0.948904
9,0.198100,0.177083,0.001800,0.695415,0.788854,0.739194,0.952105
10,0.163000,0.167411,0.001800,0.712891,0.798968,0.753479,0.954479


TrainOutput(global_step=1250, training_loss=0.22815085678100586, metrics={'train_runtime': 131.4537, 'train_samples_per_second': 152.145, 'train_steps_per_second': 9.509, 'total_flos': 139090358059872.0, 'train_loss': 0.22815085678100586, 'epoch': 25.0})

In [32]:
print("Метрики после дообучения:")
trainer.evaluate(ds_tokenized_test)

Метрики после дообучения:


{'eval_loss': 0.12861043214797974,
 'eval_model_preparation_time': 0.0018,
 'eval_precision': 0.7602459016393442,
 'eval_recall': 0.842311661506708,
 'eval_f1': 0.7991775188485265,
 'eval_accuracy': 0.9634898652901926,
 'eval_runtime': 1.4995,
 'eval_samples_per_second': 133.381,
 'eval_steps_per_second': 8.67,
 'epoch': 25.0}

### 5. Evaluation

In [33]:
predictions_output = trainer.predict(ds_tokenized_test)
logits = predictions_output.predictions
labels = predictions_output.label_ids
preds = np.argmax(logits, axis=-1)

true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
pred_labels = [[label_names[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(preds, labels)]

y_true_flat = [l for seq in true_labels for l in seq]
y_pred_flat = [l for seq in pred_labels for l in seq]

cm = confusion_matrix(y_true_flat, y_pred_flat, labels=label_names)
cm_df = pd.DataFrame(cm, index=label_names, columns=label_names)
print("Confusion Matrix:")
cm_df

Confusion Matrix:


,O,B-PER,I-PER,B-LOC,I-LOC,B-MEDIA,I-MEDIA,B-GEOPOLIT,I-GEOPOLIT,B-ORG,I-ORG
O,41919,20,68,8,34,1,1,6,0,77,296
B-PER,15,1944,36,3,1,1,0,1,0,10,1
I-PER,16,13,4743,0,5,0,3,0,0,0,42
B-LOC,7,44,1,573,5,0,0,27,0,28,4
I-LOC,12,1,109,11,668,0,2,1,0,0,72
B-MEDIA,22,5,1,1,0,224,1,2,0,31,1
I-MEDIA,34,1,18,0,3,9,186,0,0,9,102
B-GEOPOLIT,5,10,2,28,0,0,0,613,0,14,2
I-GEOPOLIT,11,0,43,2,63,0,0,1,0,0,25
B-ORG,51,5,6,27,0,4,1,7,0,1045,36


In [34]:
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels))


Classification Report:
              precision    recall  f1-score   support

    GEOPOLIT       0.85      0.84      0.85       674
         LOC       0.69      0.75      0.72       689
       MEDIA       0.78      0.69      0.73       288
         ORG       0.59      0.76      0.66      1182
         PER       0.88      0.94      0.91      2012

   micro avg       0.76      0.84      0.80      4845
   macro avg       0.76      0.80      0.77      4845
weighted avg       0.77      0.84      0.80      4845



In [35]:
def visualize_labelled_text(labelled_text: LabelledText, color_map: dict[str, str]):
    """
    Visualize the labeled data for Named Entity Recognition (NER).

    Args:
        labelled_text: An instance of LabelledText containing the text and entities.
        color_map: A color map for each entity category.

    Returns:
        None. Displays the HTML representation of the text with highlighted entities.
    """
    # Initialize the HTML string
    labelled_text2 = labelled_text
    labelled_text2.text = labelled_text.text.replace("\n", "  ")
    html_str = ""
    last_index = 0
    
    # Sort entities by the starting index
    entities = sorted(labelled_text2.entities, key=lambda x: x.start)
    for entity in entities:
        start, stop, type = entity.start, entity.stop, entity.type
        html_str += labelled_text2.text[last_index:start]  # non-entity text before the current entity
        color = color_map[type]
        html_str += (
            f'<mark style="background-color: {color}">{labelled_text2.text[start:stop]}'
            + f'<sub>({type})</sub></mark>'
        )
        last_index = stop
    html_str += labelled_text2.text[last_index:]
    
    display(HTML(html_str))  # render HTML using Jupyter Notebook

In [36]:
COLOR_MAP = {
    'GEOPOLIT': 'yellow',
    'LOC': 'lightblue',
    'MEDIA': "green",
    'PER': "cyan",
    'ORG': "red",
}

In [37]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

text = "Барак Обама назначил губернатора Юты послом США в Китае Президент США Барак Обама 16 мая назначил губернатора штата Юта Джона Хантсмена-младшего (John Huntsman Jr.) послом США в Китае, пишет The Washington Times. По словам представителя Белого дома, кандидатуру Хантсмена Обаме предложил помощник по азиатской политике Джефф Бэйдер (Jeff Bader), который представил губернатора Юты как человека, отлично знающего китайский язык, разбирающегося в проблемах региона и способного эффективно решать дипломатические задачи. Члены Республиканской партии, в том числе и бывший соперник Обамы на выборах президента США Джон Маккейн, поприветствовали выбор Обамы. Они, однако, в то же время признали, что в связи с этим назначением Хантсмен фактически лишился возможности участвовать в следующих президентских выборах. Во время предвыборной кампании 2008 года Хантсмен был одним из руководителей штаба Джона Маккейна и, по оценкам экспертов, именно он мог стать кандидатом от республиканцев на следующих выборах. Многие республиканцы отметили, что, сделав Хантсмена послом, Обама устранил потенциально опасного соперника. До избрания губернатором Юты в 2004 году Хантсмен работал торговым представителем США при Джордже Буше-младшем, а до этого был послом США в Сингапуре."
entities = ner_pipeline(text)
print(entities)

predicted_entities = [
    Ne5Span(start=ent["start"], stop=ent["end"], type=ent["entity_group"])
    for ent in entities
]

predicted_labelled_text = LabelledText(text=text, entities=predicted_entities)

visualize_labelled_text(predicted_labelled_text, COLOR_MAP)

Device set to use cuda:0


[{'entity_group': 'PER', 'score': 0.9655007, 'word': 'Барак Обама', 'start': 0, 'end': 11}, {'entity_group': 'GEOPOLIT', 'score': 0.32570925, 'word': 'Ю', 'start': 33, 'end': 34}, {'entity_group': 'PER', 'score': 0.49656078, 'word': '##ты', 'start': 34, 'end': 36}, {'entity_group': 'GEOPOLIT', 'score': 0.94108385, 'word': 'США', 'start': 44, 'end': 47}, {'entity_group': 'GEOPOLIT', 'score': 0.8766509, 'word': 'Китае', 'start': 50, 'end': 55}, {'entity_group': 'GEOPOLIT', 'score': 0.9230892, 'word': 'США', 'start': 66, 'end': 69}, {'entity_group': 'PER', 'score': 0.96309817, 'word': 'Барак Обама', 'start': 70, 'end': 81}, {'entity_group': 'PER', 'score': 0.9003958, 'word': 'Юта Джона Хантсмена - младшего ( John Huntsman Jr. )', 'start': 116, 'end': 164}, {'entity_group': 'GEOPOLIT', 'score': 0.94151914, 'word': 'США', 'start': 172, 'end': 175}, {'entity_group': 'GEOPOLIT', 'score': 0.85582006, 'word': 'Китае', 'start': 178, 'end': 183}, {'entity_group': 'MEDIA', 'score': 0.7345068, 'wor

In [38]:
trainer.save_model("./ner_rubert_tiny2")

### 6. Дообучение в MLM-режиме

In [39]:
block_size = 512

def group_texts(examples):
    """
    Группирует тексты в блоки фиксированного размера для обучения модели языкового моделирования.
    
    Эта функция объединяет все тексты из примеров в один длинный текст, затем разбивает его
    на блоки размером block_size токенов. Остаток, который меньше block_size, отбрасывается.
    
    Args:
        examples: Словарь с ключами, содержащими списки токенизированных текстов
        
    Returns:
        Словарь с теми же ключами, но значения разбиты на блоки размером block_size.
        Также добавляется ключ "labels" как копия "input_ids" для обучения MLM.
    """
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [40]:
def preprocess_function(examples):
    return tokenizer(examples["text"])

In [41]:
ds_tokenized_mlm_train = ds_tokenized_train.map(group_texts, batched=True)
ds_tokenized_mlm_test = ds_tokenized_test.map(group_texts, batched=True)

Map: 100%|██████████| 200/200 [00:00<00:00, 732.89 examples/s]


In [42]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint).to(device)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [43]:
mlm_args  = TrainingArguments(
    output_dir="./mlm_rubert_tiny2_dumps",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=25,
    weight_decay=0.01,
    seed=RANDOM_SEED
)

In [47]:
mlm_trainer = Trainer(
    model=model,
    args=mlm_args ,
    train_dataset=ds_tokenized_mlm_train,
    eval_dataset=ds_tokenized_mlm_train,  # Prevent data leakage
    data_collator=data_collator,
    processing_class=tokenizer,
    
)

In [48]:
results = mlm_trainer.evaluate(ds_tokenized_mlm_test)
print("Результаты evaluation:", results)


Результаты evaluation: {'eval_loss': 2.922074794769287, 'eval_model_preparation_time': 0.0033, 'eval_runtime': 0.8244, 'eval_samples_per_second': 132.224, 'eval_steps_per_second': 8.491}


In [49]:
mlm_trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,No log,2.788134,0.003300
2,No log,2.792165,0.003300
3,No log,2.738152,0.003300
4,No log,2.746068,0.003300
5,No log,2.730753,0.003300
6,No log,2.704244,0.003300
7,No log,2.688667,0.003300
8,No log,2.683922,0.003300
9,No log,2.676269,0.003300
10,No log,2.642901,0.003300


TrainOutput(global_step=725, training_loss=2.9608660257273707, metrics={'train_runtime': 260.3461, 'train_samples_per_second': 43.308, 'train_steps_per_second': 2.785, 'total_flos': 86047648051200.0, 'train_loss': 2.9608660257273707, 'epoch': 25.0})

In [50]:
mlm_trainer.evaluate()

{'eval_loss': 2.6501502990722656,
 'eval_model_preparation_time': 0.0033,
 'eval_runtime': 3.6096,
 'eval_samples_per_second': 124.943,
 'eval_steps_per_second': 8.034,
 'epoch': 25.0}

In [51]:
# Сохранение MLM обученной модели
print("Сохраняем MLM обученную модель...")
model.save_pretrained("./mlm_rubert_tiny2")
tokenizer.save_pretrained("./mlm_rubert_tiny2")
print("MLM модель сохранена в ./mlm_rubert_tiny2")

Сохраняем MLM обученную модель...
MLM модель сохранена в ./mlm_rubert_tiny2


### СЕКЦИЯ 3: NER AFTER MLM ОБУЧЕНИЕ

In [52]:
# Постобучение NER после MLM
model = AutoModelForTokenClassification.from_pretrained(
    "./mlm_rubert_tiny2",
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
).to(device)

tokenizer = AutoTokenizer.from_pretrained("./mlm_rubert_tiny2")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ./mlm_rubert_tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
# Пересоздаем датасеты с исправленной токенизацией
print("Создаем токенизированные датасеты")
ds_tokenized_test = make_tokenized_dataset(test_data, bio_labels_to_idx, tokenizer)
ds_tokenized_train = make_tokenized_dataset(train_data, bio_labels_to_idx, tokenizer)
print("Датасеты успешно созданы!")

Создаем токенизированные датасеты


Map: 100%|██████████| 800/800 [00:02<00:00, 336.76 examples/s]

Датасеты успешно созданы!


In [54]:
post_args  = TrainingArguments(
    output_dir="./post_mlm_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=25,
    weight_decay=0.01,
    seed=RANDOM_SEED
)

In [55]:
post_mlm_trainer = Trainer(
    model=model,
    args=post_args ,
    train_dataset=ds_tokenized_train,
    eval_dataset=ds_tokenized_test, 
    data_collator=data_collator
)

In [56]:
metric = evaluate.load("seqeval")
label_names = list(label2id.keys()) 

def compute_metrics(eval_preds: tuple[np.ndarray, np.ndarray]) -> dict[str, float]:
    """
    Compute evaluation metrics for token classification (NER) using seqeval.

    Args:
        eval_preds (tuple): A tuple containing:
            - logits (np.ndarray): Model output logits of shape (batch_size, seq_len, num_labels)
            - labels (np.ndarray): Ground truth label ids of shape (batch_size, seq_len)

    Returns:
        dict: A dictionary with overall precision, recall, F1 score, and accuracy.
    """
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    metrics = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)

    return {
        "precision": metrics["overall_precision"],
        "recall": metrics["overall_recall"],
        "f1": metrics["overall_f1"],
        "accuracy": metrics["overall_accuracy"],
    }

In [ ]:
print("Метрики до дообучения:")
post_mlm_trainer.evaluate(ds_tokenized_test)

compute_metrics(
    (post_mlm_trainer.predict(ds_tokenized_test).predictions, ds_tokenized_test['labels'])
)

Метрики до дообучения:


{'precision': 0.006213413781391725,
 'recall': 0.06418988648090815,
 'f1': 0.011330103100295093,
 'accuracy': 0.05453139332026402}

In [58]:
post_mlm_trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,No log,0.837429,0.002000
2,No log,0.513879,0.002000
3,No log,0.390324,0.002000
4,No log,0.308676,0.002000
5,No log,0.260150,0.002000
6,No log,0.228605,0.002000
7,No log,0.205675,0.002000
8,No log,0.188440,0.002000
9,No log,0.176249,0.002000
10,0.418600,0.166532,0.002000


TrainOutput(global_step=1250, training_loss=0.23091659698486328, metrics={'train_runtime': 84.2708, 'train_samples_per_second': 237.33, 'train_steps_per_second': 14.833, 'total_flos': 139090358059872.0, 'train_loss': 0.23091659698486328, 'epoch': 25.0})

In [59]:
# Финальное сохранение обученной модели после всех этапов
print("Сохраняем финально обученную модель (post-MLM NER)...")
model.save_pretrained("./final_ner_model")
tokenizer.save_pretrained("./final_ner_model")
print("Финальная модель сохранена в ./final_ner_model")

# Финальная оценка модели
print("\nФинальная оценка модели:")
final_results = post_mlm_trainer.evaluate(ds_tokenized_test)
print(f"Финальные метрики: {final_results}")

# Очистка памяти

Сохраняем финально обученную модель (post-MLM NER)...
Финальная модель сохранена в ./final_ner_model

Финальная оценка модели:


Финальные метрики: {'eval_loss': 0.1271035522222519, 'eval_model_preparation_time': 0.002, 'eval_runtime': 0.3149, 'eval_samples_per_second': 635.069, 'eval_steps_per_second': 41.279, 'epoch': 25.0}


In [ ]:
print("Метрики после дообучения:")
post_mlm_trainer.evaluate()
compute_metrics(
    (post_mlm_trainer.predict(ds_tokenized_test).predictions, ds_tokenized_test['labels'])
)

Метрики после дообучения:


{'precision': 0.7639622641509434,
 'recall': 0.8357069143446852,
 'f1': 0.7982257269590932,
 'accuracy': 0.963831585762846}

### Оценка качества модели

In [61]:
predictions_output = post_mlm_trainer.predict(ds_tokenized_test)
logits = predictions_output.predictions
labels = predictions_output.label_ids
preds = np.argmax(logits, axis=-1)

true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
pred_labels = [[label_names[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(preds, labels)]

y_true_flat = [l for seq in true_labels for l in seq]
y_pred_flat = [l for seq in pred_labels for l in seq]

cm = confusion_matrix(y_true_flat, y_pred_flat, labels=label_names)
cm_df = pd.DataFrame(cm, index=label_names, columns=label_names)
print("Confusion Matrix:")
cm_df

Confusion Matrix:


,O,B-PER,I-PER,B-LOC,I-LOC,B-MEDIA,I-MEDIA,B-GEOPOLIT,I-GEOPOLIT,B-ORG,I-ORG
O,41941,20,52,9,35,2,2,4,0,61,304
B-PER,20,1937,29,4,4,1,0,0,0,14,3
I-PER,24,15,4716,0,7,0,2,0,0,0,58
B-LOC,12,43,2,563,7,0,0,30,0,26,6
I-LOC,19,0,86,9,703,0,0,1,0,0,58
B-MEDIA,24,5,1,3,0,221,1,1,0,31,1
I-MEDIA,32,1,22,0,1,14,200,1,0,8,83
B-GEOPOLIT,8,9,2,28,0,0,0,602,0,16,9
I-GEOPOLIT,9,0,40,1,58,0,0,1,0,0,36
B-ORG,59,5,5,18,0,5,0,7,0,1033,50


In [62]:
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels))


Classification Report:
              precision    recall  f1-score   support

    GEOPOLIT       0.85      0.83      0.84       674
         LOC       0.70      0.75      0.72       689
       MEDIA       0.78      0.70      0.74       288
         ORG       0.59      0.75      0.66      1182
         PER       0.88      0.94      0.91      2012

   micro avg       0.76      0.84      0.80      4845
   macro avg       0.76      0.79      0.77      4845
weighted avg       0.77      0.84      0.80      4845



#### Visualization

In [63]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

text = "Барак Обама назначил губернатора Юты послом США в Китае Президент США Барак Обама 16 мая назначил губернатора штата Юта Джона Хантсмена-младшего (John Huntsman Jr.) послом США в Китае, пишет The Washington Times. По словам представителя Белого дома, кандидатуру Хантсмена Обаме предложил помощник по азиатской политике Джефф Бэйдер (Jeff Bader), который представил губернатора Юты как человека, отлично знающего китайский язык, разбирающегося в проблемах региона и способного эффективно решать дипломатические задачи. Члены Республиканской партии, в том числе и бывший соперник Обамы на выборах президента США Джон Маккейн, поприветствовали выбор Обамы. Они, однако, в то же время признали, что в связи с этим назначением Хантсмен фактически лишился возможности участвовать в следующих президентских выборах. Во время предвыборной кампании 2008 года Хантсмен был одним из руководителей штаба Джона Маккейна и, по оценкам экспертов, именно он мог стать кандидатом от республиканцев на следующих выборах. Многие республиканцы отметили, что, сделав Хантсмена послом, Обама устранил потенциально опасного соперника. До избрания губернатором Юты в 2004 году Хантсмен работал торговым представителем США при Джордже Буше-младшем, а до этого был послом США в Сингапуре."
entities = ner_pipeline(text)
print(entities)

predicted_entities = [
    Ne5Span(start=ent["start"], stop=ent["end"], type=ent["entity_group"])
    for ent in entities
]

predicted_labelled_text = LabelledText(text=text, entities=predicted_entities)

visualize_labelled_text(predicted_labelled_text, COLOR_MAP)

Device set to use cuda:0


[{'entity_group': 'PER', 'score': 0.9535439, 'word': 'Барак Обама', 'start': 0, 'end': 11}, {'entity_group': 'GEOPOLIT', 'score': 0.38565344, 'word': 'Ю', 'start': 33, 'end': 34}, {'entity_group': 'LOC', 'score': 0.38824052, 'word': '##ты', 'start': 34, 'end': 36}, {'entity_group': 'GEOPOLIT', 'score': 0.9504317, 'word': 'США', 'start': 44, 'end': 47}, {'entity_group': 'GEOPOLIT', 'score': 0.9099589, 'word': 'Китае', 'start': 50, 'end': 55}, {'entity_group': 'GEOPOLIT', 'score': 0.95046854, 'word': 'США', 'start': 66, 'end': 69}, {'entity_group': 'PER', 'score': 0.9685683, 'word': 'Барак Обама', 'start': 70, 'end': 81}, {'entity_group': 'PER', 'score': 0.69074476, 'word': 'Юта', 'start': 116, 'end': 119}, {'entity_group': 'PER', 'score': 0.87000597, 'word': 'Джона Хантсмена - младшего ( John Huntsman Jr. )', 'start': 120, 'end': 164}, {'entity_group': 'GEOPOLIT', 'score': 0.9474612, 'word': 'США', 'start': 172, 'end': 175}, {'entity_group': 'GEOPOLIT', 'score': 0.85961217, 'word': 'Кит

### СЕКЦИЯ 4: Использование дополнительной разметки (`synthetic_labelling.ipynb`)

### Сама секция

Перед выполнением необходимо прогнать ноутбук `synthetic_labelling.ipynb`

In [64]:
lenta = pd.read_parquet('synthetic_annotations.parquet')
lenta.head()

,words,bio_labels
0,"[Американские, фондовые, рынки, открылись, 10,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-O..."
1,"[Венесуэла, намерена, ввести, въездные, визы, ...","[B-LOC, O, O, O, O, O, O, B-LOC, O, O, O, O, B..."
2,"[Ученые, из, Австралии, и, Японии, ,, использу...","[O, O, B-LOC, O, B-LOC, O, O, O, O, O, O, O, O..."
3,"[Авианосец, нового, поколения, для, военно, -,...","[O, O, O, O, O, O, O, O, B-LOC, O, O, O, O, O,..."
4,"[Тактика, сеяния, хаоса, ,, которую, ведет, те...","[O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O, O..."


In [65]:
# Подготавливаем tokenizer и датасеты
model_checkpoint = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
lenta_dataset = Dataset.from_pandas(lenta, preserve_index=False)
labels_tokenizer_aligner = LabelsTokenizerAligner(bio_labels_to_idx, tokenizer)
tokenized_dataset = lenta_dataset.map(labels_tokenizer_aligner, batched=True, remove_columns=['words', 'bio_labels'])

Map: 100%|██████████| 9706/9706 [00:23<00:00, 416.94 examples/s]


In [66]:
print(lenta_dataset[0])


{'words': ['Американские', 'фондовые', 'рынки', 'открылись', '10', 'августа', 'резким', 'снижением', 'котировок.', 'За', 'первые', 'минуты', 'торгов', 'индекс', 'Dow', 'Jones', 'упал', 'на', '2,67', 'процента', 'и', 'снова', 'торгуется', 'ниже', 'отметки', 'в', '11', 'тысяч', 'пунктов', ',', 'S', '&', 'P', '500', 'сократился', 'на', '2,63', 'процента', 'до', '1142', 'пунктов', ',', 'а', 'Nasdaq', '-', 'на', '2,76', 'процента', 'до', '2414', 'пунктов.', 'Днем', 'ранее', 'американские', 'индексы', 'выросли', '4', '-', '5', 'процентов.', 'Таким', 'образом', 'инвесторы', 'отреагировали', 'на', 'выступление', 'главы', 'Федеральной', 'резервной', 'системы', 'Бена', 'Бернанке', ',', 'который', 'пообещал', 'сохранить', 'низкие', 'базовые', 'ставки', 'по', 'крайней', 'мере', 'до', '2013', 'года.', 'Кроме', 'того', ',', '9', 'августа', 'американские', 'рынки', 'отыгрывали', 'падение', '8', 'августа', ',', 'когда', 'биржевые', 'показатели', 'сократились', 'на', '5', '-', '6', 'процентов', 'из', '

In [67]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
).to(device)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
combined_ds = concatenate_datasets([ds_tokenized_train, tokenized_dataset])

print(combined_ds)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10506
})


In [69]:
training_args = TrainingArguments(
    output_dir="./ner_model_synthetic_dumps",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=25,
    weight_decay=0.01,
    seed=RANDOM_SEED
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=combined_ds,
    eval_dataset=ds_tokenized_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [70]:
print("Метрики до дообучения:")
trainer.evaluate(ds_tokenized_test)

Метрики до дообучения:


{'eval_loss': 2.513965129852295,
 'eval_model_preparation_time': 0.0022,
 'eval_precision': 0.002955963719966651,
 'eval_recall': 0.02414860681114551,
 'eval_f1': 0.005267185882141089,
 'eval_accuracy': 0.05510692253736443,
 'eval_runtime': 1.7447,
 'eval_samples_per_second': 114.635,
 'eval_steps_per_second': 7.451}

In [71]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Precision,Recall,F1,Accuracy
1,0.369500,0.295195,0.002200,0.468927,0.565325,0.512633,0.922735
2,0.115100,0.244491,0.002200,0.508556,0.588854,0.545768,0.931260
3,0.089100,0.227794,0.002200,0.528644,0.601858,0.562880,0.935019
4,0.063900,0.203355,0.002200,0.543773,0.612797,0.576225,0.937807
5,0.057200,0.179921,0.002200,0.569038,0.638803,0.601906,0.942285
6,0.052000,0.176716,0.002200,0.582560,0.648091,0.613581,0.943796
7,0.043100,0.161995,0.002200,0.620184,0.681115,0.649223,0.948220
8,0.040600,0.152796,0.002200,0.636977,0.701135,0.667518,0.950918
9,0.038200,0.151597,0.002200,0.649729,0.717853,0.682095,0.952519
10,0.032500,0.137759,0.002200,0.672980,0.740764,0.705247,0.956116


TrainOutput(global_step=16425, training_loss=0.04489754396667945, metrics={'train_runtime': 878.0959, 'train_samples_per_second': 299.113, 'train_steps_per_second': 18.705, 'total_flos': 1531921473032496.0, 'train_loss': 0.04489754396667945, 'epoch': 25.0})

In [72]:
print("Метрики после дообучения:")
trainer.evaluate(ds_tokenized_test)

Метрики после дообучения:


{'eval_loss': 0.1333988606929779,
 'eval_model_preparation_time': 0.0022,
 'eval_precision': 0.726063829787234,
 'eval_recall': 0.7888544891640867,
 'eval_f1': 0.756157879117618,
 'eval_accuracy': 0.96327404183378,
 'eval_runtime': 1.4714,
 'eval_samples_per_second': 135.928,
 'eval_steps_per_second': 8.835,
 'epoch': 25.0}

In [73]:
predictions_output = trainer.predict(ds_tokenized_test)
logits = predictions_output.predictions
labels = predictions_output.label_ids
preds = np.argmax(logits, axis=-1)

true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
pred_labels = [[label_names[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(preds, labels)]

y_true_flat = [l for seq in true_labels for l in seq]
y_pred_flat = [l for seq in pred_labels for l in seq]

cm = confusion_matrix(y_true_flat, y_pred_flat, labels=label_names)
cm_df = pd.DataFrame(cm, index=label_names, columns=label_names)
print("Confusion Matrix:")
cm_df

Confusion Matrix:


,O,B-PER,I-PER,B-LOC,I-LOC,B-MEDIA,I-MEDIA,B-GEOPOLIT,I-GEOPOLIT,B-ORG,I-ORG
O,41858,17,133,11,112,2,1,4,2,42,248
B-PER,7,1973,20,7,3,0,0,0,0,2,0
I-PER,35,13,4736,0,10,0,4,0,1,0,23
B-LOC,2,16,2,630,2,0,1,20,0,13,3
I-LOC,20,0,25,73,745,0,0,1,0,0,12
B-MEDIA,14,0,0,0,0,153,4,1,0,114,2
I-MEDIA,22,0,1,0,0,4,125,0,0,34,176
B-GEOPOLIT,4,3,1,259,1,0,0,399,0,2,5
I-GEOPOLIT,6,2,5,4,107,0,0,1,16,0,4
B-ORG,38,2,0,8,0,1,0,1,0,1112,20


In [74]:
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels))


Classification Report:
              precision    recall  f1-score   support

    GEOPOLIT       0.84      0.54      0.66       674
         LOC       0.49      0.78      0.60       689
       MEDIA       0.79      0.49      0.60       288
         ORG       0.63      0.81      0.71      1182
         PER       0.88      0.91      0.90      2012

   micro avg       0.73      0.79      0.76      4845
   macro avg       0.73      0.70      0.69      4845
weighted avg       0.76      0.79      0.76      4845



In [75]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

text = "Барак Обама назначил губернатора Юты послом США в Китае Президент США Барак Обама 16 мая назначил губернатора штата Юта Джона Хантсмена-младшего (John Huntsman Jr.) послом США в Китае, пишет The Washington Times. По словам представителя Белого дома, кандидатуру Хантсмена Обаме предложил помощник по азиатской политике Джефф Бэйдер (Jeff Bader), который представил губернатора Юты как человека, отлично знающего китайский язык, разбирающегося в проблемах региона и способного эффективно решать дипломатические задачи. Члены Республиканской партии, в том числе и бывший соперник Обамы на выборах президента США Джон Маккейн, поприветствовали выбор Обамы. Они, однако, в то же время признали, что в связи с этим назначением Хантсмен фактически лишился возможности участвовать в следующих президентских выборах. Во время предвыборной кампании 2008 года Хантсмен был одним из руководителей штаба Джона Маккейна и, по оценкам экспертов, именно он мог стать кандидатом от республиканцев на следующих выборах. Многие республиканцы отметили, что, сделав Хантсмена послом, Обама устранил потенциально опасного соперника. До избрания губернатором Юты в 2004 году Хантсмен работал торговым представителем США при Джордже Буше-младшем, а до этого был послом США в Сингапуре."
entities = ner_pipeline(text)
print(entities)

predicted_entities = [
    Ne5Span(start=ent["start"], stop=ent["end"], type=ent["entity_group"])
    for ent in entities
]

predicted_labelled_text = LabelledText(text=text, entities=predicted_entities)

visualize_labelled_text(predicted_labelled_text, COLOR_MAP)

Device set to use cuda:0


[{'entity_group': 'PER', 'score': 0.99825144, 'word': 'Барак Обама', 'start': 0, 'end': 11}, {'entity_group': 'LOC', 'score': 0.694159, 'word': 'Юты', 'start': 33, 'end': 36}, {'entity_group': 'GEOPOLIT', 'score': 0.9564898, 'word': 'США', 'start': 44, 'end': 47}, {'entity_group': 'GEOPOLIT', 'score': 0.95044225, 'word': 'Китае', 'start': 50, 'end': 55}, {'entity_group': 'GEOPOLIT', 'score': 0.973243, 'word': 'США', 'start': 66, 'end': 69}, {'entity_group': 'PER', 'score': 0.997154, 'word': 'Барак Обама', 'start': 70, 'end': 81}, {'entity_group': 'LOC', 'score': 0.96809644, 'word': 'Юта', 'start': 116, 'end': 119}, {'entity_group': 'PER', 'score': 0.9407775, 'word': 'Джона Хантсмена - младшего ( John Huntsman Jr. )', 'start': 120, 'end': 164}, {'entity_group': 'GEOPOLIT', 'score': 0.95686287, 'word': 'США', 'start': 172, 'end': 175}, {'entity_group': 'GEOPOLIT', 'score': 0.94162846, 'word': 'Китае', 'start': 178, 'end': 183}, {'entity_group': 'MEDIA', 'score': 0.6737799, 'word': 'The W

In [76]:
trainer.save_model("./ner_model_synthetic")

## Выводы

- MLM FineTuning перед NER дал наилучшее значение Precision: 0.764 и чуть более низкие Recall и F1 по сравнению с прямым fine‑tuning без MLM (0.842 и 0.799 соотв.).
- Accuracy везде одинаково высокая и малоприменимая для данной задачи метрика
- Добавление синтетических аннотаций заметно ухудшило все ключевые метрики (F1 упал до 0.756, Precision и Recall также на 4-5 п.п.).Вероятно, дело в низком качестве данных или несоответствия синтетики реальному распределению. В особенности из-за того, что в синтетике не присутствовали теги GEOPOLIT и MEDIA, из-за чего показатели упали для типов сущностей.


| Подход            | Precision  | Recall     | F1         | Accuracy   |
| ----------------- | ----------:| ----------:| ----------:| ----------:|
| Simple Finetuning | 0.76024590 | 0.84231166 | 0.79917751 | 0.96348986 |
| MLM FT            | 0.76396226 | 0.83570691 | 0.79822572 | 0.96383158 |
| With Synthetic    | 0.72606382 | 0.78885448 | 0.75615787 | 0.96327404 |
